# Non-Ergodic GMM Regression (Type-2) using STAN, CMDSTAN Library
## Partially Uncorrelated Anelastic Attenuation Cells

This notebook is used to estimate the non-ergodic coefficients, anelastic attenuation coefficients, and hyper-parameters of a type-2 non-ergodic GMM though Gaussian Process regression using the Bayesian software STAN. 

The Type-2 GMM compnents, with uncorrelated anelastic attenuation cells, are:
- The non-ergodic effects are modeled by:
  - $\delta c_{0}$:    constant shift of non-ergodic GMM with respect to base ergodic GMM
  - $\delta c_{1,E}$:  spatially varying eathquake adjustment as a function of the earthquake coordinates
  - $\delta c_{1a,S}$: spatially varying site adjustment as a function of the site coordinates
  - $\delta c_{1b,S}$: spatially independent site adjustment as a function of the station id
  - $c_{ca,P}$:        cell-specific anelastic attenuation
- The aleatory variability is captured by:
  - $\delta B^0_{e}$: between-event aleatory term
  - $\delta W^0_{e,s}$: within-event aleatory term
- The non-ergodic hyperparameters are:
  - $\ell_{1,E}$:     correlation lenght of spatially varying earthquake adjustment
  - $\ell_{1a,S}$:    correlation lenght of spatially varying site adjustment
  - $\omega_{1,E}$:   scale of spatially varying earthquake adjustment 
  - $\omega_{1a,S}$:  scale of spatially varying site adjustment
  - $\omega_{1b,S}$:  scale of spatially independent site adjustment
  - $\omega_{ca,P}$:  scale of anelastic attenuation cell coefficients
  - $\mu_{ca,P}$:     mean of cell-specific anelastic attenuation   
  - $\tau_{0}$:       standard deviation of between-event aleatory
  - $\phi_{0}$:       standard deviation of within-event aleatory

**Disclaimer:** The non-ergodic regression is perfomed on the total residuals (column ``res_name``) of a base ergodic GMM without including the effect of anelastic attenuation. 
The total regression residuals ($\epsilon_{tot}$) without the effect of anelastic attenuation are defined as:
$$
\epsilon_{tot} = log(IM) - ((\mu_{erg}(M,R_{rup},V_{S30}, ...) - c_{a~erg} ~ R_{rup})
$$
where $IM$ is the intesity parameter of interest (e.g. PGA, PSA), $\mu_{erg}$ is mean functional form of the ergodic GMM in log space, and $c_{a~erg}$ is the ergodic anelastic attenuation coefficient.


This script was developed as part of the Non-ergodic Methodology and Modeling Tools research project summarized in the report by Lavrentiadis G., Kuehn N., Bozorgnia Y., Seylabi E., Meng X., Goulet C., and Kottke A. (2022), "Non‐ergodic Methodology and Modeling Tools (Report GIRS-2022-04)." Natural Hazards Risk and Resiliency Research Center, University of California, Los Angeles.

Support provided by the California Department of Transportation (Caltrans) and Pacific Gas and Electric Company (PG&E) is gratefully acknowledged. 

## Load required libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#user functions
sys.path.insert(0,'../Python_lib/regression/cmdstan')
sys.path.insert(0,'../Python_lib/plotting')
import pylib_contour_plots as pycplt

## User Input, Definition of Regression Parameters 
The 1st synthetic dataset based the NGAWest2 CA metadata for the type-2 NGMM is used as an example.
The user should update the variables in this section to point to the regression dataset of interest.

The entire NGAWest3 CA syntethic dataset is loaded in this section and in the preprocessing section only the records that are part of the NGAWest2 CA are kept.

In [2]:
#regression name
reg_name  = 'example_regression_ds2'

#regression dataset
flatfile_fname = '../../Data/Flatfiles/examp_datasets/CatalogNGAWest3CALite_synthetic_data_ngmm2_small_corr_len_Y1.csv'
#cell-path info and distance matrix
cellinfo_fname = '../../Data/Flatfiles/examp_datasets/CatalogNGAWest3CALite_cellinfo.csv'
celldist_fname = '../../Data/Flatfiles/examp_datasets/CatalogNGAWest3CALite_distancematrix.csv'

#output directory
dir_out = '../../Data/Regression/example_ds2_cmdstan/'

#filename for stan regression code
# #dense matrix implementation
# sm_fname = '../Stan_lib/regression_stan_model2_uncorr_cells_unbounded_hyp_chol_efficient.stan'
# from regression_cmdstan_model2_uncorr_cells_unbounded_hyp import RunStan
#sparse matrix implementation
sm_fname = '../Stan_lib/regression_stan_model2_uncorr_cells_sparse_unbounded_hyp_chol_efficient.stan'
from regression_cmdstan_model2_uncorr_cells_sparse_unbounded_hyp import RunStan

#STAN parameters
res_name        = 'tot'
n_iter_warmup   = 500
n_iter_sampling = 500
n_chains        = 4
adapt_delta     = 0.8
max_treedepth   = 10
#parallel options
# flag_parallel = True
flag_parallel = False
#ergodic anelastic attenutation
c_a_erg = 0

## Load Files

In [3]:
#load flatfile
df_flatfile = pd.read_csv(flatfile_fname)

#load cell dataframes
df_cellinfo = pd.read_csv(cellinfo_fname)
df_celldist = pd.read_csv(celldist_fname)

## Preprocessing 
This section can be used by the user for any pre-porcessing steps of the regression dataset.

In [4]:
#keep only North records of NGAWest2
df_flatfile = df_flatfile.loc[df_flatfile.dsid==0,:]

## Run Regression

In [ ]:
print(f'Running regression for:%s'%(reg_name))

#run time start
run_t_strt = time.time()

#run stan model
RunStan(df_flatfile, df_cellinfo, df_celldist, sm_fname, 
        reg_name, dir_out, res_name, c_a_erg=c_a_erg, 
        n_iter_warmup=n_iter_warmup, n_iter_sampling=n_iter_sampling, n_chains=n_chains,        
        adapt_delta=adapt_delta, max_treedepth=max_treedepth,
        stan_parallel=flag_parallel)
       
#run time end
run_t_end = time.time()

#compute run time
run_tm = (run_t_end - run_t_strt)/60
  
#log run time
df_run_info = pd.DataFrame({'computer_name':os.uname()[1],'reg_name':reg_name,'run_time':run_tm}, 
                            index=[1])
                           
#write out run info
fname_reginfo   = '%s/run_info.csv'%(dir_out)
df_run_info.reset_index(drop=True).to_csv(fname_reginfo, index=False)

print(f'Completed regression for: %s'%(reg_name))

Running regression for:example_regression_ds2
max R_rup misfit 1.3784529073745944e-12


INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiling stan file /mnt/halcloud_nfs/glavrent/Research/NHR3_repos/ngmm_tools/Analyses/Stan_lib/regression_stan_model2_uncorr_cells_sparse_unbounded_hyp_chol_efficient.stan to exe file /mnt/halcloud_nfs/glavrent/Research/NHR3_repos/ngmm_tools/Analyses/Stan_lib/regression_stan_model2_uncorr_cells_sparse_unbounded_hyp_chol_efficient
INFO:cmdstanpy:compiled model executable: /mnt/halcloud_nfs/glavrent/Research/NHR3_repos/ngmm_tools/Analyses/Stan_lib/regression_stan_model2_uncorr_cells_sparse_unbounded_hyp_chol_efficient
--- Translating Stan model to C++ code ---
bin/stanc  --o=/mnt/halcloud_nfs/glavrent/Research/NHR3_repos/ngmm_tools/Analyses/Stan_lib/regression_stan_model2_uncorr_cells_sparse_unbounded_hyp_chol_efficient.hpp /mnt/halcloud_nfs/glavrent/Research/NHR3_repos/ngmm_tools/Analyses/Stan_lib/regression_stan_model2_uncorr_cells_sparse_unbounded_hyp_chol_efficient.stan
Warning in '/mnt/halcloud_nfs/glavrent/Researc

## Regression Review
The next code chunks produce summary plots to review the regression fit

In [ ]:
%matplotlib inline

#load non-ergodic coefficients and residuals
fname_hparam  = dir_out + reg_name + '_stan_hyperparameters.csv'
fname_gmotion = dir_out + reg_name + '_stan_residuals.csv'
fname_coeff   = dir_out + reg_name + '_stan_coefficients.csv'
fname_atten   = dir_out + reg_name + '_stan_catten.csv'

df_hparam  = pd.read_csv(fname_hparam, index_col=0)
df_gmotion = pd.read_csv(fname_gmotion, index_col=0)
df_coeff   = pd.read_csv(fname_coeff, index_col=0)
df_atten   = pd.read_csv(fname_atten, index_col=0)

#merge gm-flatfile with non-ergodic coeffs and res 
df_gmotion = pd.merge(df_flatfile[['mag','Rrup','Vs30']], df_gmotion, left_index=True, right_index=True)
df_coeff   = pd.merge(df_flatfile[['mag','Rrup','Vs30']], df_coeff,   left_index=True, right_index=True)
#merge cell atten coeffs with cell atten info
df_atten   = pd.merge(df_cellinfo, df_atten[['c_cap_mean','c_cap_med','c_cap_sig']], 
                      left_index=True, right_index=True)

#print mean and standard deviations of aleatory terms
print(f'Between-event mean:\t %.3f'%df_gmotion.res_between.mean())
print(f'Within-event mean:\t %.3f'%df_gmotion.res_within.mean())
print(f'Between-event std:\t %.3f'%df_gmotion.res_between.std())
print(f'Within-event std:\t %.3f'%df_gmotion.res_within.std())

### Non-ergodic Residuals

In [ ]:
#between-event residuals 
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_gmotion.mag, df_gmotion.res_between)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('magnitude',               fontsize=30)
ax.set_ylabel('between-event residuals', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#within-event residuals vs mag
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_gmotion.mag, df_gmotion.res_within)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('magnitude',               fontsize=30)
ax.set_ylabel('within-event residuals', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#within-event residuals vs dist
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_gmotion.Rrup, df_gmotion.res_within)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('rupture distance',       fontsize=30)
ax.set_ylabel('within-event residuals', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

### Non-ergodic Coefficients

In [ ]:
#spatially varying earthquake adjustment
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_coeff.mag, df_coeff.dc_1e_mean)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel('magnitude',              fontsize=30)
ax.set_ylabel(r'$\mu(\delta c_{1,E})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#spatially varying site adjustment
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(np.log(df_coeff.Vs30), df_coeff.dc_1as_mean)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel(r'$\ln(V_{S30}$',          fontsize=30)
ax.set_ylabel(r'$\mu(\delta c_{1a,S})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

#spatially varying site adjustment
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(np.log(df_coeff.Vs30), df_coeff.dc_1bs_mean)
ax.axhline(y=0, color='black', linestyle='--')
#figure properties
ax.set_xlabel(r'$\ln(V_{S30})$',         fontsize=30)
ax.set_ylabel(r'$\mu(\delta c_{1b,S})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

In [ ]:
#earthquake indices
_, eq_idx  = np.unique(df_coeff[['eqid']].values, axis=0, return_index=True)
#station indices
_, sta_idx = np.unique(df_coeff[['ssn']].values, axis=0, return_index=True)

#spatially varying earthquake adjustment
#---------------------
#mean
#---   ---   ---   ---
cbar_label = r'$\mu(\delta c_{1,E})$'
data2plot  = df_coeff[['eqLat','eqLon','dc_1e_mean']].values[eq_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

#epistemic uncertainty
#---   ---   ---   ---
cbar_label = r'$\psi(\delta c_{1,E})$'
data2plot  = df_coeff[['eqLat','eqLon','dc_1e_sig']].values[eq_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='Purples_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

In [ ]:
#spatially varying site adjustment
#---------------------
#mean
#---   ---   ---   ---
cbar_label = r'$\mu(\delta c_{1a,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1as_mean']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

#epistemic uncertainty
#---   ---   ---   ---
cbar_label = r'$\psi(\delta c_{1a,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1as_sig']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='Purples_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

In [ ]:
#spatially independent site adjustment
#---------------------
#mean
#---   ---   ---   ---
cbar_label = r'$\mu(\delta c_{1b,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1bs_mean']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='RdYlBu_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

#epistemic uncertainty
#---   ---   ---   ---
cbar_label = r'$\psi(\delta c_{1b,S})$'
data2plot  = df_coeff[['staLat','staLon','dc_1bs_mean']].values[sta_idx,:]
fig, ax, cbar, data_crs, _ = pycplt.PlotScatterCAMap(data2plot, cmap='Purples_r', log_cbar=False, marker_size=60,
                                                     frmt_clb = '%.2f')
#update colorbar 
cbar.ax.tick_params(tick1On=1, labelsize=30)
cbar.set_label(cbar_label, size=35)
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlabel_style = {'size': 30}
gl.ylabel_style = {'size': 30}
gl.xlocator = mticker.FixedLocator([-123, -119, -115])
gl.ylocator = mticker.FixedLocator([ 33,   37,   41])
#apply tight layout
fig.tight_layout()

### Cell-specific Anelastic Attenuation

In [ ]:
#cell-specific anelastic attenuation: mean versus std 
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(df_atten.c_cap_sig, df_atten.c_cap_mean)
ax.axhline(y=df_hparam.loc['mean','mu_cap'], color='black', linestyle='--')
#figure properties
ax.set_xlabel(r'$\psi(c_{ca,P})$', fontsize=30)
ax.set_ylabel(r'$\mu(c_{ca,P})$', fontsize=30)
ax.grid(which='both')
ax.tick_params(axis='x', labelsize=25)
ax.tick_params(axis='y', labelsize=25)
fig.tight_layout()

In [ ]:
cbar_label = r'$\mu(c_{ca,P}$)'
data2plot = df_atten[['mptLat','mptLon','c_cap_mean']].values
#create figure
fig, ax, cbar, data_crs, gl = pycplt.PlotCellsCAMapMed(data2plot)
#plot cells
#cell coordinates
cell_latlon_edge = df_cellinfo[['q5Lat','q5Lon','q6Lat','q6Lon','q8Lat','q8Lon', 
                                'q7Lat','q7Lon','q5Lat','q5Lon']].values
for ce_xy in cell_latlon_edge:
    ax.plot(ce_xy[[1,3,5,7,9]],ce_xy[[0,2,4,6,8]], color='gray', transform=data_crs)
#figure limits
#ax.set_xlim( fig_latlon_win[:,1] )
#ax.set_ylim( fig_latlon_win[:,0] )
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.top_labels   = False
gl.right_labels = False
gl.xlabel_style = {'size': 25}
gl.ylabel_style = {'size': 25}
#update colorbar 
cbar.set_label(cbar_label, size=30)
cbar.ax.tick_params(labelsize=25)
#apply tight layout
#fig.show()
fig.tight_layout()

cbar_label = r'$\psi(c_{ca,P}$)'
data2plot = df_atten[['mptLat','mptLon','c_cap_sig']].values
#create figure
fig, ax, cbar, data_crs, gl = pycplt.PlotCellsCAMapSig(data2plot)
#plot cells
#cell coordinates
cell_latlon_edge = df_cellinfo[['q5Lat','q5Lon','q6Lat','q6Lon','q8Lat','q8Lon', 
                                'q7Lat','q7Lon','q5Lat','q5Lon']].values
for ce_xy in cell_latlon_edge:
    ax.plot(ce_xy[[1,3,5,7,9]],ce_xy[[0,2,4,6,8]], color='gray', transform=data_crs)
#figure limits
#ax.set_xlim( fig_latlon_win[:,1] )
#ax.set_ylim( fig_latlon_win[:,0] )
#grid lines
gl = ax.gridlines(draw_labels=True)
gl.top_labels   = False
gl.right_labels = False
gl.xlabel_style = {'size': 25}
gl.ylabel_style = {'size': 25}
#update colorbar 
cbar.set_label(cbar_label, size=30)
cbar.ax.tick_params(labelsize=25)
#apply tight layout
#fig.show()
fig.tight_layout()